### 1.Data Preparation

At first, read the data set

In [1]:
import numpy as np


def readData(filePath: str = "") -> list:
    with open(filePath, 'r+') as f:
        dic = {"data": [], "label": []}
        tmpd = []
        tmpl = []
        for line in f.readlines():
            line = line.strip()
            if line == "-DOCSTART- -X- -X- O":
                continue
            if not line and not tmpd:
                continue
            elif not line and tmpd:
                dic["data"].append(tmpd[:])
                dic["label"].append(tmpl[:])
                tmpd.clear()
                tmpl.clear()
            else:
                word, _, _, tag = line.split(' ')
                tmpd.append(word)
                tmpl.append(tag)
    return dic

Decide padding size: choose the longest sentence in the data set
Find the label nums

In [2]:
testdata, traindata, devdata = readData('./data/test.txt'), readData('./data/train.txt'), readData('./data/valid.txt')
print("longest sentence length:",
      max([len(each) for each in testdata['data'] + traindata['data'] + devdata['data']]))

labelset = set()
for word in testdata['label'] + traindata['label'] + devdata['label']:
    for label in word:
        labelset.add(label)
print("The num of labels: ", len(labelset), labelset)

longest sentence length: 124
The num of labels:  9 {'I-PER', 'B-PER', 'I-MISC', 'I-ORG', 'B-MISC', 'B-LOC', 'O', 'I-LOC', 'B-ORG'}


In [7]:
# generate word vector
from transformers import AutoTokenizer, AutoModel
import torch

# device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
# tokenizer = AutoTokenizer.from_pretrained("nicoladecao/msmarco-word2vec256000-bert-base-uncased")
# model = AutoModel.from_pretrained("nicoladecao/msmarco-word2vec256000-bert-base-uncased")

# model.to(device)

In [4]:
import numpy as np


t1 = {}
t2 = {}
for idx, each in enumerate(labelset):
    t1[each] = idx
    t2[idx] = each

t1['PAD'] = len(t1)
t2[len(t2)] = 'PAD'

dataset = {"label2id": t1, "id2label": t2}


# def tokenize_func(dataset, label2id, tokenizer, model="", device="", MAX_SEQ_Len=124):
#     # PAD - padding, 1-startpadding,2-endpadding
#     print("Padding ID:", tokenizer.pad_token_id)
#     MAX_SEQ_Len = MAX_SEQ_Len + 2
#     data = dataset['data']
#     label = dataset['label']
#     sp = label2id["SP"]
#     ep = label2id["EP"]
#     pad = label2id["PAD"]
#     for i in range(len(data)):
#         # print(data[i])
#         s = " ".join(data[i])
#         # token = tokenizer(s, max_length=MAX_SEQ_Len, return_tensors="pt", padding='max_length')
#         # token.to(device)
#         # data[i] = model(**token).last_hidden_state
#         t = tokenizer.encode_plus(s, max_length=MAX_SEQ_Len, return_tensors="pt", padding='max_length')
#         # print(t)
#         data[i] = [t['input_ids'],t['attention_mask']]
#         label[i] = [sp] + [label2id[each] for each in label[i]] + [ep] + [pad] * (MAX_SEQ_Len - len(label[i]) - 2)
#         # print(len(data[i]),len(label[i]))

def tokenize_func(datas, label2id, word2id, MAX_SEQ_Len=124):
    # 0 - padding
    data = datas['data']
    label = datas['label']
    pad = label2id["PAD"]
    for i in range(len(data)):
        l = len(data[i])
        tokenlist = [word2id["PAD"]] * MAX_SEQ_Len
        for j in range(l):
            w = data[i][j]
            if w not in word2id:
                word2id[w] = len(word2id)
            tokenlist[j] = word2id[w]
        label[i] = [label2id[each] for each in label[i]] + [pad] * (MAX_SEQ_Len - l)
        attension_mask = [1] * l + [0] * (MAX_SEQ_Len - l)
        data[i] = [tokenlist, attension_mask]

def generate_dict(data):
    dic = {'PAD':0}
    for sen in data:
        for w in sen:
            if w not in dic:
                dic[w]=len(dic)
    return dic

# tokenize_func(traindata, dataset['label2id'], dataset['word2id'], 124)
# tokenize_func(devdata, dataset['label2id'], dataset['word2id'], 124)
# tokenize_func(testdata, dataset['label2id'], dataset['word2id'], 124)

dataset["trainset"] = traindata
dataset["devset"] = devdata
dataset["testset"] = testdata
dataset['word2id'] = generate_dict(traindata['data']+devdata['data']+testdata['data'])
dataset['longestLen'] = 124
# maybe can add word2vec later


write to file

In [5]:
import  json
with open("./Mydata.json",'w+') as f:
    json.dump(dataset,f)

In [ ]:
# loaddata
with open("./Mydata.json",'r+') as f:
    ret = json.load(f)